In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
from sklearn.linear_model import ElasticNet
from datetime import datetime
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

import time
from datetime import datetime

In [2]:
# set the options about displays
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('max_info_columns', 1001)

In [3]:
train_raw = pd.read_csv('./original/train.csv')
test_raw = pd.read_csv('./original/test.csv')
subway_raw = pd.read_csv('./original/Subways.csv')
school_raw = pd.read_csv('./original/Schools.csv')

In [4]:
train = train_raw.copy()
test = test_raw.copy()
subway = subway_raw.copy()
school = school_raw.copy()

In [5]:
train.shape

(1601458, 25)

In [6]:
test.shape

(3918, 25)

In [7]:
train = train[train['transaction_real_price'] > 60000000]

In [8]:
train = train[train['transaction_real_price'] < 3000000000]

In [9]:
train = train.assign(is_train=True )

In [10]:
test =test.assign(is_train=False )

In [11]:
all_data = pd.concat([train, test], ignore_index=False)

# SCHOOL 만져서 붙이기 

* Elementary school이 있는 법정동에 'elementary' feature를 추가 후 개수 입력   
* Private school이 있는 법정동에 'private' feature를 추가 후 개수 입력

In [12]:
school.head(10)

,school_code,latitude,longitude,school_class,operation_type,highschool_type,gender,foundation_date,address_by_law
0,S000003511,37.491,127.015,elementary,national,NaN,both,1953.1.31,1165010800
1,S000003563,37.578,127.003,elementary,national,NaN,both,1946.8.22,1111016800
2,S010000737,37.481,127.059,elementary,public,NaN,both,1982.9.20,1168010300
3,S010000738,37.486,127.058,elementary,public,NaN,both,1987.11.17,1168010300
4,S010000741,37.481,127.052,elementary,public,NaN,both,1983.11.30,1168010300
5,S010000742,37.508,127.026,elementary,public,NaN,both,1972.11.23,1168010800
6,S010000743,37.494,127.065,elementary,public,NaN,both,1981.9.9,1168010600
7,S010000744,37.493,127.054,elementary,public,NaN,both,1979.3.27,1168011800
8,S010000745,37.481,127.086,elementary,public,NaN,both,1994.8.5,1168011400
9,S010000746,37.464,127.106,elementary,public,NaN,both,1932.11.25,1168011100


In [13]:
school['elementary']  = school['school_class'].map(lambda x : 1 if x == 'elementary' else 0)

In [14]:
school['operation_type'].unique()

array(['national', 'public', 'private'], dtype=object)

In [15]:
school['private'] = school['operation_type'].map(lambda x : 1 if x == 'private' else 0)

In [16]:
school_m = school.groupby(by = 'address_by_law').sum().reset_index()

In [17]:
school_m.head()

,address_by_law,latitude,longitude,elementary,private
0,1111010100,150.350,507.882,1,0
1,1111011300,112.735,380.901,1,2
2,1111013200,37.576,126.988,1,1
3,1111013400,37.575,126.988,1,0
4,1111014100,37.578,126.983,0,1


In [18]:
aa = school.groupby(by = 'address_by_law').count().reset_index()

In [19]:
a = aa.iloc[:, 0: 2]

In [20]:
school_mm = pd.merge(school_m, a, on='address_by_law', how='left')

In [21]:
school_mm.head(10)

,address_by_law,latitude,longitude,elementary,private,school_code
0,1111010100,150.350,507.882,1,0,4
1,1111011300,112.735,380.901,1,2,3
2,1111013200,37.576,126.988,1,1,1
3,1111013400,37.575,126.988,1,0,1
4,1111014100,37.578,126.983,0,1,1
5,1111014500,37.578,126.982,0,1,1
6,1111014600,37.580,126.986,1,0,1
7,1111014800,112.752,380.962,0,3,3
8,1111016200,37.573,127.003,1,0,1
9,1111016800,75.156,254.007,1,0,2


In [22]:
school_mm['mean_latitude']  = school_mm['latitude'] / school_mm['school_code']
school_mm['mean_longitude']  = school_mm['longitude'] / school_mm['school_code']

In [23]:
school_mm = school_mm.drop(columns=['latitude', 'longitude'])

In [24]:
school_mm = school_mm.rename(columns={'school_code':'num_schools'})

In [25]:
school_mm

,address_by_law,elementary,private,num_schools,mean_latitude,mean_longitude
0,1111010100,1,0,4,37.588,126.970
1,1111011300,1,2,3,37.578,126.967
2,1111013200,1,1,1,37.576,126.988
3,1111013400,1,0,1,37.575,126.988
4,1111014100,0,1,1,37.578,126.983
5,1111014500,0,1,1,37.578,126.982
6,1111014600,1,0,1,37.580,126.986
7,1111014800,0,3,3,37.584,126.987
8,1111016200,1,0,1,37.573,127.003
9,1111016800,1,0,2,37.578,127.003


In [26]:
all_data = pd.merge(all_data, school_mm, on='address_by_law', how='left')

In [27]:
all_data.head(10)

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price,is_train,elementary,private,num_schools,mean_latitude,mean_longitude
0,0,5584,1,200601,11~20,1999,47.430,6,37.586,127.000,1111017100,163.000,136,1,8.000,4.000,individual,gas,91120,65.630,46,1.000,1.000,corridor,215000000,True,nan,nan,nan,nan,nan
1,1,5584,1,200601,11~20,1999,44.370,8,37.586,127.000,1111017100,163.000,136,1,8.000,4.000,individual,gas,91119,61.390,10,2.000,1.000,corridor,200000000,True,nan,nan,nan,nan,nan
2,2,5059,1,200601,11~20,1992,54.700,8,37.581,127.014,1111017400,902.000,585,5,14.000,9.000,individual,gas,8430,72.360,201,2.000,1.000,corridor,168000000,True,2.000,1.000,3.000,37.579,127.014
3,3,2816,1,200601,11~20,1993,64.660,11,37.580,127.012,1111017400,902.000,919,7,15.000,11.000,individual,gas,5839,87.300,284,2.000,1.000,corridor,165000000,True,2.000,1.000,3.000,37.579,127.014
4,4,2816,1,200601,11~20,1993,106.620,7,37.580,127.012,1111017400,902.000,919,7,15.000,11.000,individual,gas,5836,127.740,112,4.000,2.000,stairway,280000000,True,2.000,1.000,3.000,37.579,127.014
5,5,2815,1,200601,11~20,2000,84.920,9,37.575,126.961,1111018700,1365.000,964,12,23.000,10.000,individual,gas,5831,109.880,454,3.000,2.000,stairway,415000000,True,1.000,0.000,1.000,37.574,126.960
6,6,2815,1,200601,11~20,2000,60.000,13,37.575,126.961,1111018700,1365.000,964,12,23.000,10.000,individual,gas,5833,84.330,207,3.000,1.000,corridor,267000000,True,1.000,0.000,1.000,37.574,126.960
7,7,9867,1,200601,11~20,2005,84.760,10,37.559,127.020,1114016200,652.000,461,9,23.000,6.000,individual,gas,11862,104.300,82,3.000,2.000,stairway,415000000,True,6.000,7.000,16.000,37.560,127.014
8,8,2818,1,200601,11~20,1999,84.880,18,37.555,127.014,1114016200,2091.000,2282,19,20.000,8.000,individual,gas,5843,107.650,576,3.000,2.000,stairway,310000000,True,6.000,7.000,16.000,37.560,127.014
9,9,2817,1,200601,11~20,2002,59.940,12,37.550,127.009,1114016200,4329.000,5150,42,18.000,11.000,individual,gas,5842,85.900,864,3.000,1.000,corridor,319000000,True,6.000,7.000,16.000,37.560,127.014


# SUBWAY

* 유의하다고 생각되는 subwayline 유무를 dummy변수로 생성(서울 : 1,2,3,4,9,분당,신분당, 부산 : 4호선,동해선)
* transfer line feature 생성

In [28]:
subway.head()

,station_id,latitude,longitude,subway_line,address_by_law
0,1,37.556,126.972,"1,4,KJ,AP",1114012000.000
1,2,37.566,126.977,"1,2",1114016700.000
2,3,37.570,126.983,1,1111012600.000
3,4,37.572,126.992,"1,3,5",1111015600.000
4,5,37.571,127.002,1,1111016300.000


In [29]:
subway['subway_line'] = subway['subway_line'].str.split(',')

subway['transfer_subway'] = subway['subway_line'].str.len()

station_g = subway.groupby(by='station_id').mean().reset_index()

In [30]:
station_g.info()

count_line = subway.copy()

def line_distict(arr, line):
    if line in arr :
        dap = 1
    else:
        dap = 0
    return dap

subway['subway_line'][0]

subway['sub_line2'] = subway['subway_line'].map(lambda x : line_distict(x, '2'))
subway['sub_line4'] = subway['subway_line'].map(lambda x : line_distict(x, '4'))
subway['sub_line1'] = subway['subway_line'].map(lambda x : line_distict(x, '1'))
subway['sub_line3'] = subway['subway_line'].map(lambda x : line_distict(x, '3'))
subway['sub_lineBD'] = subway['subway_line'].map(lambda x : line_distict(x, 'BD'))
subway['sub_lineB4'] = subway['subway_line'].map(lambda x : line_distict(x, 'B4'))
subway['sub_lineDL'] = subway['subway_line'].map(lambda x : line_distict(x, 'DL'))
subway['sub_line9'] = subway['subway_line'].map(lambda x : line_distict(x, '9'))
subway['sub_lineND'] = subway['subway_line'].map(lambda x : line_distict(x, 'ND'))

subway.head(10)

subway_m1 = subway.groupby(['address_by_law']).sum().reset_index()

temp_list = ['sub_line2',
 'sub_line4',
 'sub_line1',
 'sub_lineBD',
 'sub_lineB4',
 'sub_line3',
 'sub_lineDL',
 'sub_line9',
 'sub_lineND']

for i in temp_list:
    subway_m1[i] = subway_m1[i].map(lambda x : 1 if x != 0 else 0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405 entries, 0 to 404
Data columns (total 5 columns):
station_id         405 non-null int64
latitude           405 non-null float64
longitude          405 non-null float64
address_by_law     396 non-null float64
transfer_subway    405 non-null int64
dtypes: float64(3), int64(2)
memory usage: 15.9 KB


In [31]:
subway_c1 = subway.groupby(['address_by_law']).count().reset_index()

In [32]:
c1 = subway_c1.iloc[:, 0:2]

In [33]:
subway_mm = pd.merge(subway_m1, c1, on='address_by_law', how='left')

In [34]:
subway_mm.head(100)

,address_by_law,station_id_x,latitude,longitude,transfer_subway,sub_line2,sub_line4,sub_line1,sub_line3,sub_lineBD,sub_lineB4,sub_lineDL,sub_line9,sub_lineND,station_id_y
0,1111010700.000,68,37.576,126.973,1,0,0,0,1,0,0,0,0,0,1
1,1111011600.000,341,37.572,126.976,1,0,0,0,0,0,0,0,0,0,1
2,1111012600.000,3,37.570,126.983,1,0,0,1,0,0,0,0,0,0,1
3,1111014100.000,69,37.577,126.985,1,0,0,0,1,0,0,0,0,0,1
4,1111015600.000,4,37.572,126.992,3,0,0,1,1,0,0,0,0,0,1
5,1111016300.000,5,37.571,127.002,1,0,0,1,0,0,0,0,0,0,1
6,1111017200.000,102,37.582,127.002,1,0,1,0,0,0,0,0,0,0,1
7,1111017400.000,381,37.580,127.015,1,0,0,0,0,0,0,0,0,0,1
8,1111017500.000,10,37.573,127.016,2,0,0,1,0,0,0,0,0,0,1
9,1114010400.000,11,37.566,126.982,1,1,0,0,0,0,0,0,0,0,1


In [35]:
subway_mm['mean_latitude']  = subway_mm['latitude'] / subway_mm['station_id_y']
subway_mm['mean_longitude']  = subway_mm['longitude'] / subway_mm['station_id_y']

In [36]:
trr = subway_mm.columns.tolist()

In [37]:
subway_final = subway_mm.drop(columns=['station_id_x','longitude','latitude',])
subway_final=subway_final.rename(columns={'station_id_y' : 'num_subways'})

In [38]:
temp = all_data.copy()

In [39]:
all_data = pd.merge(all_data, subway_final, on='address_by_law', how='left')

In [40]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1583201 entries, 0 to 1583200
Data columns (total 44 columns):
key                                   1583201 non-null int64
apartment_id                          1583201 non-null int64
city                                  1583201 non-null int64
transaction_year_month                1583201 non-null int64
transaction_date                      1583201 non-null object
year_of_completion                    1583201 non-null int64
exclusive_use_area                    1583201 non-null float64
floor                                 1583201 non-null int64
latitude                              1583201 non-null float64
longitude                             1583201 non-null float64
address_by_law                        1583201 non-null int64
total_parking_capacity_in_site        1496170 non-null float64
total_household_count_in_sites        1583201 non-null int64
apartment_building_count_in_sites     1583201 non-null int64
tallest_building_in_site

# Feature Engneering

### 1. Split the year and month of transaction

In [41]:
all_data['transcation_year'] = all_data['transaction_year_month'].map(lambda x : int(str(x)[:4]))
all_data['transcation_month'] = all_data['transaction_year_month'].map(lambda x : int(str(x)[4:]))

### 2. Rename columns

In [42]:
all_data = all_data.rename(columns={'mean_latitude_x' : 'mean_latitude_schools','mean_longitude_x':'mean_longitude_schools','mean_latitude_y' : 'mean_latitude_subways','mean_longitude_y':'mean_longitude_subways',})

In [43]:
all_data.drop(columns=['transaction_year_month'], inplace=True)

In [44]:
all_data.head()

,key,apartment_id,city,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price,is_train,elementary,private,num_schools,mean_latitude_schools,mean_longitude_schools,transfer_subway,sub_line2,sub_line4,sub_line1,sub_line3,sub_lineBD,sub_lineB4,sub_lineDL,sub_line9,sub_lineND,num_subways,mean_latitude_subways,mean_longitude_subways,transcation_year,transcation_month
0,0,5584,1,11~20,1999,47.430,6,37.586,127.000,1111017100,163.000,136,1,8.000,4.000,individual,gas,91120,65.630,46,1.000,1.000,corridor,215000000,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2006,1
1,1,5584,1,11~20,1999,44.370,8,37.586,127.000,1111017100,163.000,136,1,8.000,4.000,individual,gas,91119,61.390,10,2.000,1.000,corridor,200000000,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2006,1
2,2,5059,1,11~20,1992,54.700,8,37.581,127.014,1111017400,902.000,585,5,14.000,9.000,individual,gas,8430,72.360,201,2.000,1.000,corridor,168000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1
3,3,2816,1,11~20,1993,64.660,11,37.580,127.012,1111017400,902.000,919,7,15.000,11.000,individual,gas,5839,87.300,284,2.000,1.000,corridor,165000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1
4,4,2816,1,11~20,1993,106.620,7,37.580,127.012,1111017400,902.000,919,7,15.000,11.000,individual,gas,5836,127.740,112,4.000,2.000,stairway,280000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1


### 3. Labelling transaction date to 1, 2, 3,

In [45]:
all_data['transaction_date'].unique()

array(['11~20', '1~10', '21~31', '21~28', '21~30', '21~29'], dtype=object)

In [46]:
data_map = {'1~10':1,'11~20':2,'21~31':3, '21~28':3, '21~30':3, '21~29':3}

In [47]:
all_data['transaction_date'] = all_data['transaction_date'].map(data_map)

In [48]:
all_data.head()

,key,apartment_id,city,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price,is_train,elementary,private,num_schools,mean_latitude_schools,mean_longitude_schools,transfer_subway,sub_line2,sub_line4,sub_line1,sub_line3,sub_lineBD,sub_lineB4,sub_lineDL,sub_line9,sub_lineND,num_subways,mean_latitude_subways,mean_longitude_subways,transcation_year,transcation_month
0,0,5584,1,2,1999,47.430,6,37.586,127.000,1111017100,163.000,136,1,8.000,4.000,individual,gas,91120,65.630,46,1.000,1.000,corridor,215000000,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2006,1
1,1,5584,1,2,1999,44.370,8,37.586,127.000,1111017100,163.000,136,1,8.000,4.000,individual,gas,91119,61.390,10,2.000,1.000,corridor,200000000,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2006,1
2,2,5059,1,2,1992,54.700,8,37.581,127.014,1111017400,902.000,585,5,14.000,9.000,individual,gas,8430,72.360,201,2.000,1.000,corridor,168000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1
3,3,2816,1,2,1993,64.660,11,37.580,127.012,1111017400,902.000,919,7,15.000,11.000,individual,gas,5839,87.300,284,2.000,1.000,corridor,165000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1
4,4,2816,1,2,1993,106.620,7,37.580,127.012,1111017400,902.000,919,7,15.000,11.000,individual,gas,5836,127.740,112,4.000,2.000,stairway,280000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1


In [49]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1583201 entries, 0 to 1583200
Data columns (total 45 columns):
key                                   1583201 non-null int64
apartment_id                          1583201 non-null int64
city                                  1583201 non-null int64
transaction_date                      1583201 non-null int64
year_of_completion                    1583201 non-null int64
exclusive_use_area                    1583201 non-null float64
floor                                 1583201 non-null int64
latitude                              1583201 non-null float64
longitude                             1583201 non-null float64
address_by_law                        1583201 non-null int64
total_parking_capacity_in_site        1496170 non-null float64
total_household_count_in_sites        1583201 non-null int64
apartment_building_count_in_sites     1583201 non-null int64
tallest_building_in_sites             1583191 non-null float64
lowest_building_in_site

# Missing Values

In [50]:
all_data.isnull().sum()

key                                        0
apartment_id                               0
city                                       0
transaction_date                           0
year_of_completion                         0
exclusive_use_area                         0
floor                                      0
latitude                                   0
longitude                                  0
address_by_law                             0
total_parking_capacity_in_site         87031
total_household_count_in_sites             0
apartment_building_count_in_sites          0
tallest_building_in_sites                 10
lowest_building_in_sites                  10
heat_type                               2045
heat_fuel                               9711
room_id                                    0
supply_area                                0
total_household_count_of_area_type         0
room_count                               696
bathroom_count                           696
front_door

In [51]:
all_data.head(10)

,key,apartment_id,city,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price,is_train,elementary,private,num_schools,mean_latitude_schools,mean_longitude_schools,transfer_subway,sub_line2,sub_line4,sub_line1,sub_line3,sub_lineBD,sub_lineB4,sub_lineDL,sub_line9,sub_lineND,num_subways,mean_latitude_subways,mean_longitude_subways,transcation_year,transcation_month
0,0,5584,1,2,1999,47.430,6,37.586,127.000,1111017100,163.000,136,1,8.000,4.000,individual,gas,91120,65.630,46,1.000,1.000,corridor,215000000,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2006,1
1,1,5584,1,2,1999,44.370,8,37.586,127.000,1111017100,163.000,136,1,8.000,4.000,individual,gas,91119,61.390,10,2.000,1.000,corridor,200000000,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2006,1
2,2,5059,1,2,1992,54.700,8,37.581,127.014,1111017400,902.000,585,5,14.000,9.000,individual,gas,8430,72.360,201,2.000,1.000,corridor,168000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1
3,3,2816,1,2,1993,64.660,11,37.580,127.012,1111017400,902.000,919,7,15.000,11.000,individual,gas,5839,87.300,284,2.000,1.000,corridor,165000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1
4,4,2816,1,2,1993,106.620,7,37.580,127.012,1111017400,902.000,919,7,15.000,11.000,individual,gas,5836,127.740,112,4.000,2.000,stairway,280000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1
5,5,2815,1,2,2000,84.920,9,37.575,126.961,1111018700,1365.000,964,12,23.000,10.000,individual,gas,5831,109.880,454,3.000,2.000,stairway,415000000,True,1.000,0.000,1.000,37.574,126.960,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2006,1
6,6,2815,1,2,2000,60.000,13,37.575,126.961,1111018700,1365.000,964,12,23.000,10.000,individual,gas,5833,84.330,207,3.000,1.000,corridor,267000000,True,1.000,0.000,1.000,37.574,126.960,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2006,1
7,7,9867,1,2,2005,84.760,10,37.559,127.020,1114016200,652.000,461,9,23.000,6.000,individual,gas,11862,104.300,82,3.000,2.000,stairway,415000000,True,6.000,7.000,16.000,37.560,127.014,5.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,3.000,37.556,127.011,2006,1
8,8,2818,1,2,1999,84.880,18,37.555,127.014,1114016200,2091.000,2282,19,20.000,8.000,individual,gas,5843,107.650,576,3.000,2.000,stairway,310000000,True,6.000,7.000,16.000,37.560,127.014,5.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,3.000,37.556,127.011,2006,1
9,9,2817,1,2,2002,59.940,12,37.550,127.009,1114016200,4329.000,5150,42,18.000,11.000,individual,gas,5842,85.900,864,3.000,1.000,corridor,319000000,True,6.000,7.000,16.000,37.560,127.014,5.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,3.000,37.556,127.011,2006,1


In [52]:
all_data.to_csv('./csv/190127_dataset.csv', index=False)

In [53]:
a_sub =all_data['mean_latitude_schools'].dropna()
print(max(a_sub))
b_scho =all_data['mean_longitude_schools'].dropna()
print(max(b_scho))

all_data['mean_latitude_schools'] = all_data['mean_latitude_schools'].fillna(max(a_sub)+0.1)
all_data['mean_longitude_schools'] = all_data['mean_longitude_schools'].fillna(max(b_scho)+0.1)

37.681620091666666
129.226985


In [54]:
a_sub =all_data['mean_latitude_subways'].dropna()
print(max(a_sub))
b_scho =all_data['mean_longitude_subways'].dropna()
print(max(b_scho))

all_data['mean_latitude_subways'] = all_data['mean_latitude_subways'].fillna(max(a_sub)+0.1)
all_data['mean_longitude_subways'] = all_data['mean_longitude_subways'].fillna(max(b_scho)+0.1)

37.68984604
129.20236210000002


In [55]:
train = all_data[all_data['is_train'] == True]

In [61]:
all_data.isnull().sum()

all_data['total_parking_capacity_in_site'] = all_data['total_parking_capacity_in_site'].fillna(0)
all_data['num_schools'] = all_data['num_schools'].fillna(0)
all_data['num_subways'] = all_data['num_subways'].fillna(0)
all_data['transfer_subway'] = all_data['transfer_subway'].fillna(0)

all_data.isnull().sum()

all_data['elementary'] = all_data['elementary'].fillna(0)
all_data['private'] = all_data['private'].fillna(0)

all_data.isnull().sum()

fill_mode = ['tallest_building_in_sites','lowest_building_in_sites','heat_type','heat_fuel','room_count','bathroom_count','front_door_structure']
for i in fill_mode:
    all_data[i] = all_data[i].fillna(all_data[i].mode()[0])

ssan_zip = all_data[all_data['transaction_real_price'] < 100000000]

fill_la_long = ['mean']


sub_arr = []
for i in list(all_data.columns):
    if i[:3] == 'sub':
        sub_arr.append(i)

for i in sub_arr:
    all_data[i] = all_data[i].fillna(0)


In [62]:
all_data = all_data.fillna(0)

all_data['address_by_law'] = all_data['address_by_law'].map(lambda x : str(x)[:6])
all_data.head()

,key,apartment_id,city,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price,is_train,elementary,private,num_schools,mean_latitude_schools,mean_longitude_schools,transfer_subway,sub_line2,sub_line4,sub_line1,sub_line3,sub_lineBD,sub_lineB4,sub_lineDL,sub_line9,sub_lineND,num_subways,mean_latitude_subways,mean_longitude_subways,transcation_year,transcation_month
0,0,5584,1,2,1999,47.430,6,37.586,127.000,111101,163.000,136,1,8.000,4.000,individual,gas,91120,65.630,46,1.000,1.000,corridor,215000000,True,0.000,0.000,0.000,37.782,129.327,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37.790,129.302,2006,1
1,1,5584,1,2,1999,44.370,8,37.586,127.000,111101,163.000,136,1,8.000,4.000,individual,gas,91119,61.390,10,2.000,1.000,corridor,200000000,True,0.000,0.000,0.000,37.782,129.327,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37.790,129.302,2006,1
2,2,5059,1,2,1992,54.700,8,37.581,127.014,111101,902.000,585,5,14.000,9.000,individual,gas,8430,72.360,201,2.000,1.000,corridor,168000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1
3,3,2816,1,2,1993,64.660,11,37.580,127.012,111101,902.000,919,7,15.000,11.000,individual,gas,5839,87.300,284,2.000,1.000,corridor,165000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1
4,4,2816,1,2,1993,106.620,7,37.580,127.012,111101,902.000,919,7,15.000,11.000,individual,gas,5836,127.740,112,4.000,2.000,stairway,280000000,True,2.000,1.000,3.000,37.579,127.014,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,37.580,127.015,2006,1


In [63]:
all_data.isnull().sum().sum()

0

In [70]:
all_data['transaction_real_price'] =np.log(all_data['transaction_real_price'])

/home/pirl/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


# Modeling

In [64]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1583201 entries, 0 to 1583200
Data columns (total 45 columns):
key                                   1583201 non-null int64
apartment_id                          1583201 non-null int64
city                                  1583201 non-null int64
transaction_date                      1583201 non-null int64
year_of_completion                    1583201 non-null int64
exclusive_use_area                    1583201 non-null float64
floor                                 1583201 non-null int64
latitude                              1583201 non-null float64
longitude                             1583201 non-null float64
address_by_law                        1583201 non-null object
total_parking_capacity_in_site        1583201 non-null float64
total_household_count_in_sites        1583201 non-null int64
apartment_building_count_in_sites     1583201 non-null int64
tallest_building_in_sites             1583201 non-null float64
lowest_building_in_sit

In [65]:
all_data = all_data.drop(['key'],axis =1)
train = all_data[all_data['is_train']==True]
test = all_data[all_data['is_train']!=True]

In [67]:
print(all_data.shape)
print(train.shape)
print(test.shape)

(1583201, 44)
(1579283, 44)
(3918, 44)


In [68]:
train = train.drop('is_train', axis=1)
test = test.drop('is_train', axis=1)

In [69]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = train.drop(columns=['transaction_real_price'])
y = train['transaction_real_price']

## Train set 예측
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1105498, 42)
(1105498,)
(473785, 42)
(473785,)


In [ ]:
#모델 선언
rf = RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [ ]:

# Fitting the model
print(datetime.today())       #현재 날짜 가져오기
start_vect=time.time()
#===============================================================
rf.fit(X_train, y_train)
#===============================================================
print("training Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

In [ ]:
print(datetime.today())            # 현재 날짜 가져오기

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(np.expm1(y_pred), np.exp(y_test))

print('MSE : {:,.6f}'.format(mse))